## Model Variables

In [1]:
IMAGE_SIZE = 224
BATCH_SIZE = 32
DATASET_SIZE = 1500
MAX_EPOCHS = 20
K_FOLDS = 5

## Data Preparation

In [2]:
# import required libraries
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil
from sklearn.metrics import confusion_matrix, classification_report
from datetime import datetime
from packaging import version
%reload_ext tensorboard
import tensorboard
from sklearn.model_selection import StratifiedKFold
from tensorboard.plugins.hparams import api as hp
import time

print("Tensorflow Version:", tf.__version__)
print("Tensorboard Version:", tensorboard.__version__)
print("GPU Found:", tf.test.is_gpu_available())

Tensorflow Version: 1.15.5
Tensorboard Version: 1.15.0
GPU Found: True


In [3]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([64, 128]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.1, 0.2]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
HP_L_RATE= hp.HParam('learning_rate', hp.Discrete([0.0005, 0.001]))

METRIC_ACCURACY = 'accuracy'

with tf.compat.v1.summary.FileWriter('./logs/hparam_tuning'):
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER, HP_L_RATE],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [4]:
# read in train data
train_df = pd.read_csv('../covidx-cxr2/train_COVIDx9B.txt', 
                       sep=" ", header=None)
# add columns to go from 0, 1, 2, 3 to patient id, filename, class etc
train_df.columns=['patient id', 'filename', 'class', 'data source']
# drop patient id and datasource as not needed
train_df=train_df.drop(['patient id', 'data source'], axis=1 )

# read in test data
test_df = pd.read_csv('../covidx-cxr2/test_COVIDx9B.txt', 
                      sep=" ", header=None)
# add columns to go from 0, 1, 2, 3 to patient id, filename, class etc
test_df.columns=['patient id', 'filename', 'class', 'data source']
# drop patient id and datasource as not needed
test_df=test_df.drop(['patient id', 'data source'], axis=1 )

In [5]:
print("Train class counts:")
print(train_df['class'].value_counts())
print("\nTest class counts:")
print(test_df['class'].value_counts())

Train class counts:
positive    16490
negative    13992
Name: class, dtype: int64

Test class counts:
positive    200
negative    200
Name: class, dtype: int64


In [6]:
negative  = train_df[train_df['class']=='negative']   # normal values in class column
positive = train_df[train_df['class']=='positive']  # COVID-19 values in class column

from sklearn.utils import resample
# downsample training data to 400 values of each class, to reduce class bias and reduce training time

df_negative_downsampled = resample(negative, replace = True, n_samples = DATASET_SIZE//2)
df_positive_downsampled = resample(positive, replace = True, n_samples = DATASET_SIZE//2) 

#concatenate
train_df = pd.concat([df_negative_downsampled, df_positive_downsampled])

from sklearn.utils import shuffle
train_df = shuffle(train_df) # shuffling so that there is particular sequence
print("Train class counts:")
print(train_df['class'].value_counts())

Train class counts:
positive    750
negative    750
Name: class, dtype: int64


In [7]:
print("Train class counts:")
print(train_df['class'].value_counts())
print("\nTest class counts:")
print(test_df['class'].value_counts())

Train class counts:
positive    750
negative    750
Name: class, dtype: int64

Test class counts:
positive    200
negative    200
Name: class, dtype: int64


In [8]:
# preprocess images
test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input)

test_gen = test_datagen.flow_from_dataframe(dataframe = test_df, directory="../covidx-cxr2/test", x_col='filename', 
                                            y_col='class', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, 
                                            color_mode='rgb', class_mode='binary', shuffle=False)

Found 400 validated image filenames belonging to 2 classes.


In [9]:
def preprocess_images_cv(train_index, val_index):
    training_data = train_df.iloc[train_index]
    validation_data = train_df.iloc[val_index]
    train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input,
                                       rotation_range = 20, width_shift_range = 0.2, height_shift_range = 0.2, 
                                       shear_range = 0.2, zoom_range = 0.1, horizontal_flip = True)

    #Now fit the them to get the images from directory (name of the images are given in dataframe) with augmentation

    train_gen = train_datagen.flow_from_dataframe(dataframe = training_data, directory="../covidx-cxr2/train", x_col='filename', 
                                                  y_col='class', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, 
                                                  color_mode='rgb', class_mode='binary')
    valid_gen = test_datagen.flow_from_dataframe(dataframe = validation_data, directory="../covidx-cxr2/train", x_col='filename',
                                                 y_col='class', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, 
                                                 color_mode='rgb', class_mode='binary')
    return train_gen, valid_gen


## ResNet50V2

In [10]:
# required libraries for ResNet
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

In [11]:
def create_model(hparams):
    # create the base pre-trained model
    base_model = keras.applications.ResNet50V2(weights='imagenet', input_shape = (IMAGE_SIZE,IMAGE_SIZE,3),
                                                  include_top=False)

    model = keras.Sequential([ base_model, 
                                 keras.layers.GlobalAveragePooling2D(), 
                                 keras.layers.Dense(hparams[HP_NUM_UNITS], activation='relu'),
                                 keras.layers.BatchNormalization(), 
                                 keras.layers.Dropout(hparams[HP_DROPOUT]),
                                 keras.layers.Dense(1, activation='sigmoid')
                                ])
    
    optimizer_name = hparams[HP_OPTIMIZER]
    learning_rate = hparams[HP_L_RATE]
    if optimizer_name == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        raise ValueError("unexpected optimizer name: %r" % (optimizer_name,))


    model.compile(optimizer = optimizer,
                  loss = 'binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [12]:
def get_model_name(k):
    return str("ResNet50V2-Binary-" + "Fold-"+str(k)+"-"+datetime.now().strftime("%Y%m%d-%H%M%S"))

In [13]:
# perform HP tuning and CV on the model on the data
Y = train_df['class']
trained_models = []

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in HP_DROPOUT.domain.values:
        for optimizer in HP_OPTIMIZER.domain.values:
            for learning_rate in HP_L_RATE.domain.values:
                
                skf = StratifiedKFold(n_splits = K_FOLDS, shuffle = True) 
                k_i = 0
                k_fold_models = []
                elapsed_times = []
                hparams = {
                    HP_NUM_UNITS: num_units,
                    HP_DROPOUT: dropout_rate,
                    HP_OPTIMIZER: optimizer,
                    HP_L_RATE: learning_rate
                }
                print('HPARAMS:',{h.name: hparams[h] for h in hparams})
                
                for train_index, val_index in skf.split(np.zeros(len(Y)),Y):
                    print('--- Starting trial: %s' % k_i)
                    t = time.process_time()
                    train_gen, valid_gen = preprocess_images_cv(train_index, val_index)

                    logdir = os.path.join('logs','fit', get_model_name(k_i))
                    hpdir = os.path.join('logs','hparam_tuning', get_model_name(k_i))
                    modeldir = os.path.join('trained classifiers',str(get_model_name(k_i)+".h5"))
                    callbacks = [
                        keras.callbacks.ModelCheckpoint(modeldir, save_best_only=True, verbose = 0),
                        keras.callbacks.EarlyStopping(patience=3, monitor='val_loss', verbose=1, restore_best_weights=True),
                        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, verbose=1),
                        keras.callbacks.TensorBoard(log_dir=logdir)
                      ]
                   
                    hp.hparams(hparams)  # record the values used in this trial
                    model = create_model(hparams)
                    model.fit(train_gen, validation_data=valid_gen, epochs=MAX_EPOCHS, callbacks=callbacks)
                    elapsed_times.append(time.process_time() - t)
                    k_fold_models.append(modeldir)
                    
                    del model, train_gen, valid_gen
                    keras.backend.clear_session()
                    k_i += 1
                    
                trained_models.append([hparams, k_fold_models, elapsed_times])

HPARAMS: {'num_units': 64, 'dropout': 0.1, 'optimizer': 'adam', 'learning_rate': 0.0005}
--- Starting trial: 0
Found 1200 validated image filenames belonging to 2 classes.
Found 300 validated image filenames belonging to 2 classes.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/20
38/38 [==============================] - 44s 1s/step - loss: 0.4038 - acc: 0.8325 - val_loss: 1.7968 - val_acc: 0.7900
Epoch 2/20
38/38 [==============================] - 30s 795ms/step - loss: 0.2436 - acc: 0.9042 - val_loss: 1.7603 - val_acc: 0.8267
Epoch 3/20
38/38 [==============================] - 30s 794ms/step - loss: 0.2110 - acc: 0.9183 - val_loss: 1.3904 - val_acc: 0.8200
Epoch 4/20
38/38 [==============================] - 30s 789ms/step - loss: 0.1948 - acc: 0.9250 - val_loss: 0.8058 - val_acc: 0.8600
Epoch 5/20
38/38 [==============================] - 30s 792ms/st

Epoch 14/20
10/38 [======>.......................] - ETA: 10s - loss: 0.2150 - acc: 0.9367Restoring model weights from the end of the best epoch.

Epoch 00014: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.
38/38 [==============================] - 30s 797ms/step - loss: 0.0531 - acc: 0.9817 - val_loss: 0.2150 - val_acc: 0.9367
Epoch 00014: early stopping
13/13 [==============================] - 18s 1s/step - loss: 0.0955 - acc: 0.9675
--- Starting trial: 2
Found 1200 validated image filenames belonging to 2 classes.
Found 300 validated image filenames belonging to 2 classes.
Epoch 1/20
38/38 [==============================] - 43s 1s/step - loss: 0.4370 - acc: 0.8083 - val_loss: 3.3278 - val_acc: 0.5567
Epoch 2/20
10/38 [======>.......................] - ETA: 11s - loss: 7.2188 - acc: 0.6967
Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
38/38 [==============================] - 30s 797ms/step - loss: 0.2496 - acc: 0.9042 - val_loss: 

10/38 [======>.......................] - ETA: 11s - loss: 0.4851 - acc: 0.7767
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
38/38 [==============================] - 30s 799ms/step - loss: 0.2182 - acc: 0.9108 - val_loss: 0.4851 - val_acc: 0.7767
Epoch 7/20
38/38 [==============================] - 31s 805ms/step - loss: 0.1894 - acc: 0.9267 - val_loss: 0.3742 - val_acc: 0.8567
Epoch 8/20
10/38 [======>.......................] - ETA: 11s - loss: 0.5436 - acc: 0.9133
Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
38/38 [==============================] - 30s 785ms/step - loss: 0.1906 - acc: 0.9275 - val_loss: 0.5436 - val_acc: 0.9133
Epoch 9/20
10/38 [======>.......................] - ETA: 11s - loss: 0.4795 - acc: 0.8233
Epoch 00009: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
38/38 [==============================] - 30s 787ms/step - loss: 0.1457 - acc: 0.9500 - val_loss: 0.4795 - val_acc: 0.8233
Ep

Epoch 13/20
10/38 [======>.......................] - ETA: 12s - loss: 0.2601 - acc: 0.9200Restoring model weights from the end of the best epoch.

Epoch 00013: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
38/38 [==============================] - 31s 826ms/step - loss: 0.1108 - acc: 0.9625 - val_loss: 0.2601 - val_acc: 0.9200
Epoch 00013: early stopping
13/13 [==============================] - 19s 1s/step - loss: 0.1258 - acc: 0.9525
--- Starting trial: 4
Found 1200 validated image filenames belonging to 2 classes.
Found 300 validated image filenames belonging to 2 classes.
Epoch 1/20
38/38 [==============================] - 43s 1s/step - loss: 0.4693 - acc: 0.8250 - val_loss: 69.2705 - val_acc: 0.5000
Epoch 2/20
38/38 [==============================] - 30s 794ms/step - loss: 0.3311 - acc: 0.8733 - val_loss: 2.0439 - val_acc: 0.7367
Epoch 3/20
10/38 [======>.......................] - ETA: 10s - loss: 15.0433 - acc: 0.5300
Epoch 00003: ReduceLROnPlateau reducing lea

Epoch 6/20
38/38 [==============================] - 30s 778ms/step - loss: 0.4195 - acc: 0.7958 - val_loss: 0.4673 - val_acc: 0.7733
Epoch 7/20
38/38 [==============================] - 29s 774ms/step - loss: 0.3911 - acc: 0.8325 - val_loss: 0.4259 - val_acc: 0.7867
Epoch 8/20
38/38 [==============================] - 30s 779ms/step - loss: 0.3586 - acc: 0.8433 - val_loss: 0.3916 - val_acc: 0.8133
Epoch 9/20
38/38 [==============================] - 30s 779ms/step - loss: 0.3436 - acc: 0.8492 - val_loss: 0.3506 - val_acc: 0.8267
Epoch 10/20
38/38 [==============================] - 30s 779ms/step - loss: 0.3447 - acc: 0.8525 - val_loss: 0.3346 - val_acc: 0.8567
Epoch 11/20
38/38 [==============================] - 29s 774ms/step - loss: 0.3187 - acc: 0.8625 - val_loss: 0.3269 - val_acc: 0.8533
Epoch 12/20
38/38 [==============================] - 30s 777ms/step - loss: 0.3058 - acc: 0.8775 - val_loss: 0.3055 - val_acc: 0.8633
Epoch 13/20
38/38 [==============================] - 29s 774ms/ste

...

Epoch 00013: early stopping
13/13 [==============================] - 19s 1s/step - loss: 0.0968 - acc: 0.9675
--- Starting trial: 1
Found 1200 validated image filenames belonging to 2 classes.
Found 300 validated image filenames belonging to 2 classes.
Epoch 1/20
38/38 [==============================] - 44s 1s/step - loss: 0.4858 - acc: 0.8033 - val_loss: 14.9262 - val_acc: 0.5000
Epoch 2/20
10/38 [======>.......................] - ETA: 11s - loss: 80.6345 - acc: 0.7933
Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
38/38 [==============================] - 30s 802ms/step - loss: 0.3371 - acc: 0.8708 - val_loss: 80.6345 - val_acc: 0.7933
Epoch 3/20
38/38 [==============================] - 31s 804ms/step - loss: 0.2674 - acc: 0.8950 - val_loss: 4.8764 - val_acc: 0.8633
Epoch 4/20
38/38 [==============================] - 31s 806ms/step - loss: 0.2371 - acc: 0.9000 - val_loss: 0.4775 - val_acc: 0.8633
Epoch 5/20
10/38 [======>.......................] - ETA:

38/38 [==============================] - 31s 817ms/step - loss: 0.2810 - acc: 0.8942 - val_loss: 4.8975 - val_acc: 0.5000
Epoch 4/20
38/38 [==============================] - 31s 814ms/step - loss: 0.2461 - acc: 0.9075 - val_loss: 1.4521 - val_acc: 0.7533
Epoch 5/20
10/38 [======>.......................] - ETA: 12s - loss: 17.0187 - acc: 0.5000
Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
38/38 [==============================] - 30s 802ms/step - loss: 0.2820 - acc: 0.8833 - val_loss: 17.0187 - val_acc: 0.5000
Epoch 6/20
10/38 [======>.......................] - ETA: 12s - loss: 2.3999 - acc: 0.6033
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
38/38 [==============================] - 30s 798ms/step - loss: 0.2394 - acc: 0.9167 - val_loss: 2.3999 - val_acc: 0.6033
Epoch 7/20
38/38 [==============================] - 31s 817ms/step - loss: 0.2118 - acc: 0.9217 - val_loss: 1.0221 - val_acc: 0.7200
Epoch 8/20
38/38 [========

Epoch 13/20
10/38 [======>.......................] - ETA: 11s - loss: 0.3243 - acc: 0.8400
Epoch 00013: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
38/38 [==============================] - 30s 788ms/step - loss: 0.3024 - acc: 0.8683 - val_loss: 0.3243 - val_acc: 0.8400
Epoch 14/20
10/38 [======>.......................] - ETA: 11s - loss: 0.3113 - acc: 0.8533Restoring model weights from the end of the best epoch.

Epoch 00014: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
38/38 [==============================] - 31s 811ms/step - loss: 0.2701 - acc: 0.8950 - val_loss: 0.3113 - val_acc: 0.8533
Epoch 00014: early stopping
13/13 [==============================] - 19s 1s/step - loss: 0.5420 - acc: 0.7200
--- Starting trial: 1
Found 1200 validated image filenames belonging to 2 classes.
Found 300 validated image filenames belonging to 2 classes.
Epoch 1/20
38/38 [==============================] - 39s 1s/step - loss: 0.7879 - acc: 0.5675 - val_loss: 0

Epoch 5/20
38/38 [==============================] - 30s 796ms/step - loss: 0.3733 - acc: 0.8358 - val_loss: 0.4808 - val_acc: 0.7833
Epoch 6/20
38/38 [==============================] - 30s 797ms/step - loss: 0.3662 - acc: 0.8442 - val_loss: 0.4485 - val_acc: 0.7800
Epoch 7/20
38/38 [==============================] - 30s 794ms/step - loss: 0.3368 - acc: 0.8492 - val_loss: 0.4291 - val_acc: 0.8067
Epoch 8/20
38/38 [==============================] - 30s 796ms/step - loss: 0.3463 - acc: 0.8525 - val_loss: 0.3738 - val_acc: 0.8233
Epoch 9/20
38/38 [==============================] - 30s 796ms/step - loss: 0.3355 - acc: 0.8608 - val_loss: 0.3345 - val_acc: 0.8600
Epoch 10/20
38/38 [==============================] - 30s 796ms/step - loss: 0.3131 - acc: 0.8642 - val_loss: 0.3132 - val_acc: 0.8500
Epoch 11/20
38/38 [==============================] - 30s 794ms/step - loss: 0.3002 - acc: 0.8733 - val_loss: 0.2878 - val_acc: 0.8833
Epoch 12/20
38/38 [==============================] - 30s 796ms/step

Epoch 12/20
38/38 [==============================] - 30s 784ms/step - loss: 0.2732 - acc: 0.8875 - val_loss: 0.2445 - val_acc: 0.9200
Epoch 13/20
10/38 [======>.......................] - ETA: 10s - loss: 0.2707 - acc: 0.9267
Epoch 00013: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
38/38 [==============================] - 30s 778ms/step - loss: 0.2749 - acc: 0.8900 - val_loss: 0.2707 - val_acc: 0.9267
Epoch 14/20
38/38 [==============================] - 30s 788ms/step - loss: 0.2550 - acc: 0.9033 - val_loss: 0.2391 - val_acc: 0.9267
Epoch 15/20
10/38 [======>.......................] - ETA: 10s - loss: 0.2696 - acc: 0.9233
Epoch 00015: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
38/38 [==============================] - 30s 777ms/step - loss: 0.2267 - acc: 0.9083 - val_loss: 0.2696 - val_acc: 0.9233
Epoch 16/20
10/38 [======>.......................] - ETA: 10s - loss: 0.2715 - acc: 0.9133
Epoch 00016: ReduceLROnPlateau reducing learning rate to

## Evaluation

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from statistics import mean

In [15]:
for j in trained_models:
    model = create_model(j[0])
    accuracies = []
    k_accuracies = []
    precisions = []
    recalls = []
    f1s = []
    for i in j[1]:
        model.load_weights(i)
        test_pred = model.predict(test_gen)
        y_pred = np.rint(test_pred).flatten()
        y_test= [test_gen.class_indices[k] for k in test_df['class'].values.tolist()]
        k_accuracies.append(model.evaluate(test_gen)[1])
        accuracies.append(accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred, zero_division=0))
        recalls.append(recall_score(y_test, y_pred))
        f1s.append(f1_score(y_test, y_pred))

    print("SCORE FOR HPARAMS:", {h.name: j[0][h] for h in j[0]})
    print("Mean accuracy:", mean(accuracies))
    print("Mean k_accuracy;", mean(k_accuracies))
    print("Mean precision:", mean(precisions))
    print("Mean recall:", mean(recalls))
    print("Mean f1:", mean(f1s))
    print("Mean training time:", mean(j[2]))
    hdict = {h.name: j[0][h] for h in j[0]}
    df =  pd.DataFrame(np.array([[str("ResNet50V2-"+datetime.now().strftime("%Y%m%d-%H%M%S")),
                                  hdict.get('num_units'), hdict.get('dropout'), hdict.get('optimizer'), 
                                  hdict.get('learning_rate'),IMAGE_SIZE, BATCH_SIZE, DATASET_SIZE, MAX_EPOCHS, K_FOLDS,
                                  mean(accuracies), mean(k_accuracies), mean(precisions), mean(recalls), mean(f1s), mean(j[2])
                                 ]]),
                       columns=['Model Name','num_units', 'dropout','optimizer','learning_rate',
                                'IMAGE_SIZE', 'BATCH_SIZE', 'DATASET_SIZE', 'MAX_EPOCHS',
                                'K_FOLDS','Accuracy', 'K_accuracy', 'Precision', 'Recall', 'f1 Score', 'Training Time'])
    with open('results.csv', 'a') as f:
        df.to_csv(f, mode='a', header=f.tell()==0, index = False)

13/13 [==============================] - 19s 1s/step - loss: 0.0366 - acc: 0.9875
SCORE FOR HPARAMS: {'num_units': 64, 'dropout': 0.1, 'optimizer': 'adam', 'learning_rate': 0.0005}
Mean accuracy: 0.961
Mean k_accuracy; 0.96099997
Mean precision: 0.9674021636820483
Mean recall: 0.9550000000000001
Mean f1: 0.9607531286306098
13/13 [==============================] - 19s 1s/step - loss: 0.1456 - acc: 0.9550
SCORE FOR HPARAMS: {'num_units': 64, 'dropout': 0.1, 'optimizer': 'adam', 'learning_rate': 0.001}
Mean accuracy: 0.943
Mean k_accuracy; 0.94299996
Mean precision: 0.9478797525131388
Mean recall: 0.938
Mean f1: 0.9428242417510629
13/13 [==============================] - 19s 1s/step - loss: 0.5373 - acc: 0.7525
SCORE FOR HPARAMS: {'num_units': 64, 'dropout': 0.1, 'optimizer': 'sgd', 'learning_rate': 0.0005}
Mean accuracy: 0.774
Mean k_accuracy; 0.774
Mean precision: 0.8483258632202557
Mean recall: 0.665
Mean f1: 0.7437756075419306
13/13 [==============================] - 19s 1s/step - los

13/13 [==============================] - 21s 2s/step - loss: 0.1636 - acc: 0.9475
SCORE FOR HPARAMS: {'num_units': 128, 'dropout': 0.2, 'optimizer': 'adam', 'learning_rate': 0.001}
Mean accuracy: 0.9595
Mean k_accuracy; 0.95949996
Mean precision: 0.9304616472554895
Mean recall: 0.994
Mean f1: 0.9609756847253018
13/13 [==============================] - 21s 2s/step - loss: 0.5138 - acc: 0.7650
SCORE FOR HPARAMS: {'num_units': 128, 'dropout': 0.2, 'optimizer': 'sgd', 'learning_rate': 0.0005}
Mean accuracy: 0.751
Mean k_accuracy; 0.751
Mean precision: 0.8138613307143143
Mean recall: 0.65
Mean f1: 0.7208283769134319
13/13 [==============================] - 21s 2s/step - loss: 0.4742 - acc: 0.8000
SCORE FOR HPARAMS: {'num_units': 128, 'dropout': 0.2, 'optimizer': 'sgd', 'learning_rate': 0.001}
Mean accuracy: 0.8265
Mean k_accuracy; 0.82649994
Mean precision: 0.8922192109331549
Mean recall: 0.743
Mean f1: 0.8097750010112615


NOTE: Accuracy and loss graphs can be seen on TensorBoard, run "%tensorboard --logdir logs --host 0.0.0.0" to open